# How to build an AI Agent with DSPy and the ClickHouse MCP Server

In this notebook we'll see how to build an [DSPy](https://dspy.ai/tutorials/mcp/) AI agent that can interact with [ClickHouse's SQL playground](https://sql.clickhouse.com/) using [ClickHouse's MCP Server](https://github.com/ClickHouse/mcp-clickhouse).


## Install libraries
We need to install the DSPy library.

In [ ]:
!pip install -q --upgrade pip

In [1]:
!pip install -q dspy
!pip install -q mcp

## Setup credentials
Let's provide our Anthropic API key.

In [3]:
import os, getpass

In [4]:
os.environ["ANTHROPIC_API_KEY"] = getpass.getpass("Enter Anthropic API Key:")

Enter Anthropic API Key: ········


We'll also define the credentials to connect to the ClickHouse SQL playground:

In [5]:
env = {
    "CLICKHOUSE_HOST": "sql-clickhouse.clickhouse.com",
    "CLICKHOUSE_PORT": "8443",
    "CLICKHOUSE_USER": "demo",
    "CLICKHOUSE_PASSWORD": "",
    "CLICKHOUSE_SECURE": "true"
}

## Initialize MCP Server

Lets configure the ClickHouse MCP Server to point at the ClickHouse SQL playground.

In [6]:
from mcp import ClientSession, StdioServerParameters
from mcp.client.stdio import stdio_client
import dspy

In [15]:
server_params = StdioServerParameters(
    command="uv",
    args=[
        'run',
        '--with', 'mcp-clickhouse',
        '--python', '3.13',
        'mcp-clickhouse'
    ],
    env=env
)

## Initialize LLM
Next, let's initialize our Claude Sonnet model

In [8]:
dspy.configure(lm=dspy.LM("anthropic/claude-sonnet-4-20250514"))

## Run agent
Finally, we'll initialize and run the agent:

In [10]:
class DataAnalyst(dspy.Signature):
    """You are a data analyst. You'll be asked questions and you need to try to answer them using the tools you have access to. """

    user_request: str = dspy.InputField()
    process_result: str = dspy.OutputField(
        desc=(
            "Answer to the query"
        )
    )

In [26]:
from utils import print_dspy_result

In [27]:
async with stdio_client(server_params) as (read, write):
    async with ClientSession(read, write) as session:
        await session.initialize()
        tools = await session.list_tools()

        dspy_tools = []
        for tool in tools.tools:
            dspy_tools.append(dspy.Tool.from_mcp_tool(session, tool))

        react = dspy.ReAct(DataAnalyst, tools=dspy_tools)
        result = await react.acall(user_request="What's the most popular Amazon product category")
        print_dspy_result(result)

🤖 DSPy ReAct Result

📍 STEP 1
----------------------------------------
🧠 THINKING: I need to find information about Amazon product categories and determine which one is most popular. First, I should explore what databases are available to see if there's any Amazon-related data.

🔧 TOOL: list_databases

📊 RESULT:
   amazon
bluesky
country
covid
default
dns
environmental
food
forex
geo
git
github
hackernews
imdb
logs
metrica
mgbench
mta
noaa
nyc_taxi
nypd
ontime
opensky
otel
otel_v2
pypi
random
reddit
rubygems
sta...


📍 STEP 2
----------------------------------------
🧠 THINKING: Great! I can see there's an "amazon" database available. This is exactly what I need to find information about Amazon product categories. Let me explore the tables in the amazon database to see what data is available.

🔧 TOOL: list_tables
   Args: {'database': 'amazon'}

📊 RESULT:
   {
  "database": "amazon",
  "name": "amazon_reviews",
  "comment": "",
  "columns": [
    {
      "name": "review_date",
      "ty